In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

DB_PASSWORD = os.getenv('REDSHIFT_PASS')

In [ ]:
%%writefile dwh.cfg
[CLUSTER]
HOST='redshift-cluster-1.c2ryw0xl3tli.us-east-1.redshift.amazonaws.com'
DB_NAME='dev'
DB_USER='awsuser'
PORT=5439

[S3]
LOG_DATA='s3://udacity-dend-redshift/log-data'
LOG_JSONPATH='s3://udacity-dend-redshift/log_json_path.json'
SONG_DATA='s3://udacity-dend-redshift/song_data'
ARN='arn:aws:iam::390354360073:role/service-role/AmazonRedshift-CommandsAccessRole-20220617T184245'

Overwriting dwh.cfg


In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('dwh.cfg')

ARN = config.get("S3", "ARN")
LOG_DATA = config.get("S3", "LOG_DATA")
LOG_JSONPATH = config.get("S3", "LOG_JSONPATH")
SONG_DATA = config.get("S3", "SONG_DATA")

HOST = config.get("CLUSTER", "HOST")
DB_NAME = config.get("CLUSTER", "DB_NAME")
DB_USER = config.get("CLUSTER", "DB_USER")
PORT = config.get("CLUSTER", "PORT")

In [ ]:
# DROP TABLES

staging_events_table_drop = "DROP TABLE IF EXISTS staging_events"
staging_songs_table_drop = "DROP TABLE IF EXISTS staging_songs"
songplay_table_drop = "DROP TABLE IF EXISTS songplays"
user_table_drop = "DROP TABLE IF EXISTS users"
song_table_drop = "DROP TABLE IF EXISTS songs"
artist_table_drop = "DROP TABLE IF EXISTS artists"
time_table_drop = "DROP TABLE IF EXISTS time"

# CREATE TABLES

staging_events_table_create= ("""
    CREATE TABLE IF NOT EXISTS staging_events (
        artist VARCHAR,
        auth VARCHAR,
        firstName VARCHAR,
        gender CHAR(1),
        itemInSession INT,
        lastName VARCHAR,
        length FLOAT,
        level VARCHAR,
        location TEXT,
        method VARCHAR,
        page VARCHAR,
        registration VARCHAR,
        sessionId INT,
        song VARCHAR,
        status INT,
        ts BIGINT,
        userAgent TEXT,
        userId INT
    )
""")

staging_songs_table_create = ("""
    CREATE TABLE IF NOT EXISTS staging_songs (
        artist_id VARCHAR,
        artist_latitude FLOAT,
        artist_location TEXT,
        artist_longitude FLOAT,
        artist_name VARCHAR,
        duration FLOAT,
        num_songs INT,
        song_id VARCHAR,
        title VARCHAR,
        year INT
    )
""")

songplay_table_create = ("""
    CREATE TABLE IF NOT EXISTS songplays (
        songplay_id INT IDENTITY(0,1),
        start_time TIMESTAMP,
        user_id INT,
        level VARCHAR,
        song_id VARCHAR,
        artist_id VARCHAR,
        session_id INT,
        location TEXT,
        user_agent TEXT
    )
""")

user_table_create = ("""
    CREATE TABLE IF NOT EXISTS users (
        user_id INT,
        first_name VARCHAR,
        last_name VARCHAR,
        gender CHAR(1),
        level VARCHAR
    )
""")

song_table_create = ("""
    CREATE TABLE IF NOT EXISTS songs (
        song_id VARCHAR,
        title VARCHAR,
        artist_id VARCHAR,
        year INT,
        duration FLOAT
    )
""")

artist_table_create = ("""
    CREATE TABLE IF NOT EXISTS artists (
        artist_id VARCHAR,
        name VARCHAR,
        location TEXT ,
        latitude FLOAT ,
        longitude FLOAT
    )
""")

time_table_create = ("""
    CREATE TABLE IF NOT EXISTS time (
        start_time TIMESTAMP,
        hour INT,
        day INT,
        week INT,
        month INT,
        year INT,
        weekday VARCHAR
    )
""")

# STAGING TABLES

staging_events_copy = ("""
    copy staging_events
    from {0}
    iam_role {1}
    json {2};
""").format(LOG_DATA, ARN, LOG_JSONPATH)

staging_songs_copy = ("""
    copy staging_songs
    from {0}
    iam_role {1}
    json 'auto';
""").format(SONG_DATA, ARN)

# FINAL TABLES

songplay_table_insert = ("""
    INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
    SELECT DISTINCT 
        timestamp with time zone 'epoch' + se.ts/1000 * interval '1 second', se.userId, se.level, 
        ss.song_id, ss.artist_id, se.sessionId, se.location, se.userAgent
    FROM staging_events AS se INNER JOIN staging_songs AS ss
    ON se.song = ss.title AND se.artist = ss.artist_name AND se.length = ss.duration
    WHERE se.page = 'NextSong'
    
""")

user_table_insert = ("""
    INSERT INTO users (user_id, first_name, last_name, gender, level)
    SELECT DISTINCT userId, firstName, lastName, gender, level
    FROM staging_events
    WHERE page = 'NextSong' AND userId IS NOT NULL
""")

song_table_insert = ("""
    INSERT INTO songs (song_id, title, artist_id, year, duration)
    SELECT DISTINCT song_id, title, artist_id, year, duration
    FROM staging_songs
    WHERE song_id IS NOT NULL
""")

artist_table_insert = ("""
    INSERT INTO artists (artist_id, name, location, latitude, longitude)
    SELECT DISTINCT artist_id, artist_name, artist_location, artist_latitude, artist_longitude
    FROM staging_songs
    WHERE artist_id IS NOT NULL
""")

time_table_insert = ("""
    INSERT INTO time (start_time, hour, day, week, month, year, weekday)
    SELECT start_time, 
           extract(hour from start_time), 
           extract(day from start_time), 
           extract(week from start_time), 
           extract(month from start_time), 
           extract(year from start_time), 
           extract(weekday from start_time)
    FROM songplays
""")

# QUERY LISTS

create_table_queries = [staging_events_table_create, staging_songs_table_create, songplay_table_create, user_table_create, song_table_create, artist_table_create, time_table_create]
drop_table_queries = [staging_events_table_drop, staging_songs_table_drop, songplay_table_drop, user_table_drop, song_table_drop, artist_table_drop, time_table_drop]
copy_table_queries = [staging_events_copy, staging_songs_copy]
insert_table_queries = [songplay_table_insert, user_table_insert, song_table_insert, artist_table_insert, time_table_insert]

In [ ]:
import psycopg2

In [ ]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [ ]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [ ]:
def main():
    conn = psycopg2.connect(f"host={HOST} dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD} port={PORT}")
    cur = conn.cursor()

    drop_tables(cur, conn)
    create_tables(cur, conn)

    conn.close()


if __name__ == "__main__":
    main()

In [ ]:
def load_staging_tables(cur, conn):
    for query in copy_table_queries:
        cur.execute(query)
        conn.commit()

In [ ]:
def insert_tables(cur, conn):
    for query in insert_table_queries:
        cur.execute(query)
        conn.commit()

In [ ]:
def main():
    conn = psycopg2.connect(f"host={HOST} dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD} port={PORT}")
    cur = conn.cursor()
    
    load_staging_tables(cur, conn)
    insert_tables(cur, conn)

    conn.close()


if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd

conn = psycopg2.connect(f"host={HOST} dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD} port={PORT}")

In [ ]:
query = "SELECT DISTINCT tablename FROM pg_table_def WHERE schemaname = 'public';"

pd.read_sql_query(query, conn)

,tablename
0,artists
1,category
2,date
3,event
4,listing
5,sales
6,songplays
7,songs
8,staging_events
9,staging_songs


In [ ]:
query = "SELECT * FROM staging_events LIMIT 5;"

pd.read_sql_query(query, conn)

,artist,auth,firstname,gender,iteminsession,lastname,length,level,location,method,page,registration,sessionid,song,status,ts,useragent,userid
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.99220,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796,184,Am I High (Feat. Malice),200,1541121934796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
1,None,Logged In,Stefany,F,0,White,NaN,free,"Lubbock, TX",GET,Home,1540708070796,82,None,200,1541122176796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
2,Death Cab for Cutie,Logged In,Stefany,F,1,White,216.42404,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,A Lack Of Color (Album Version),200,1541122241796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
3,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,I Have A Wish,200,1541122457796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
4,Skillet,Logged In,Kevin,M,0,Arellano,178.02404,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,153,Monster (Album Version),200,1541126568796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66


In [ ]:
query = "SELECT * FROM staging_songs LIMIT 5;"

pd.read_sql_query(query, conn)

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,None,California - LA,None,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0
1,ARKRRTF1187B9984DA,None,,None,Sonora Santanera,177.47546,1,SOXVLOJ12AB0189215,Amor De Cabaret,0
2,ARXR32B1187FB57099,None,,None,Gob,209.60608,1,SOFSOCN12A8C143F5D,Face the Ashes,2007
3,ARD0S291187B9B7BF5,None,Ohio,None,Rated R,114.78159,1,SOMJBYD12A6D4F8557,Keepin It Real (Skit),0
4,AR8ZCNI1187B9A069B,None,,None,Planet P Project,269.81832,1,SOIAZJW12AB01853F1,Pink World,1984


In [ ]:
query = "SELECT * FROM songplays LIMIT 5;"

pd.read_sql_query(query, conn)

,songplay_id,start_time,user_id,level,song_id,artist_id,session_id,location,user_agent
0,0,2018-11-21 21:56:47,15,paid,SOZCTXZ12AB0182364,AR5KOSW1187FB35FF4,818,"Chicago-Naperville-Elgin, IL-IN-WI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5..."


In [ ]:
query = "SELECT * FROM users LIMIT 5;"

pd.read_sql_query(query, conn)

,user_id,first_name,last_name,gender,level
0,83,Stefany,White,F,free
1,48,Marina,Sutton,F,free
2,17,Makinley,Jones,F,free
3,89,Kynnedi,Sanchez,F,free
4,44,Aleena,Kirby,F,paid


In [ ]:
query = "SELECT * FROM songs LIMIT 5;"

pd.read_sql_query(query, conn)

,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179
1,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007,209.60608
2,SOIAZJW12AB01853F1,Pink World,AR8ZCNI1187B9A069B,1984,269.81832
3,SOMUYGI12AB0188633,La Culpa,ARL7K851187B99ACD2,0,226.35057
4,SOUQQEA12A8C134B1B,High Tide,ARIG6O41187B988BDD,0,228.59710


In [ ]:
query = "SELECT * FROM artists LIMIT 5;"

pd.read_sql_query(query, conn)

,artist_id,name,location,latitude,longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN
1,ARXR32B1187FB57099,Gob,,NaN,NaN
2,AR8ZCNI1187B9A069B,Planet P Project,,NaN,NaN
3,ARL7K851187B99ACD2,Andy Andy,,NaN,NaN
4,ARIG6O41187B988BDD,Richard Souther,United States,37.16793,-95.84502


In [ ]:
query = "SELECT * FROM time LIMIT 5;"

pd.read_sql_query(query, conn)

,start_time,hour,day,week,month,year,weekday
0,2018-11-21 21:56:47,21,21,47,11,2018,3
